# Clustering on Figueroa data

Aims:
- reproduce the Figueroa patient clusters (16 clusters) and assign the same labels as in Figueroa 2009
- for each cluster, obtain signature genes and plot the heatmap
- determine the best strategy for clustering the gene probes

In [1]:
# load RNAseq and methylation data
rnaseq <- read.csv('../data/Figueroa/data/GEP-filtered.tsv', sep='\t', stringsAsFactor=F,
                       header=T)
methylation <- read.csv('../data/Figueroa/data/methylation-filtered.tsv', sep='\t', stringsAsFactor=F,
                       header=T)

Warning message in file(file, "rt"):
“cannot open file '../data/Figueroa/data/GEP-filtered.tsv': No such file or directory”

ERROR: Error in file(file, "rt"): cannot open the connection
